(data-transform)=
# Data Transformation

## Introduction

It's very rare that data arrive in exactly the right form you need. Often, you'll need to create some new variables or summaries, or maybe you just want to rename the variables or reorder the observations to make the data a little easier to work with.

You'll learn how to do all that (and more!) in this chapter, which will introduce you to data transformation using the **pandas** package and a new dataset on flights that departed New York City in 2013.

The goal of this chapter is to give you an overview of all the key tools for transforming a data frame, a special kind of object that holds tabular data.

We'll come back these functions in more detail in later chapters, as we start to dig into specific types of data (e.g. numbers, strings, dates).

### Prerequisites

In this chapter we'll focus on the **pandas** package, one of the most widely used tools for data science. You'll need to ensure you have **pandas** installed. To do this, you can run

In [ ]:
import pandas as pd

If this command fails, you don't have **pandas** installed. Open up the terminal in Visual Studio Code (Terminal -> New Terminal) and type in `conda install pandas`.

Furthemore, if you wish to check which version of **pandas** you're using, it's

In [ ]:
pd.__version__

You'll also need the data. Most of the time, data will need to be loaded from a file or the internet. These data are no different, but one of the amazing things about **pandas** is how many different types of data it can load, including from files on the internet.

The data is around 50MB in size so you will need a good internet connection or a little patience for it to download.

Let's download the data:

In [ ]:
url = "https://raw.githubusercontent.com/byuidatascience/data4python4ds/master/data-raw/flights/flights.csv"
flights = pd.read_csv(url)


If the above code worked, then you've downloaded the data in CSV format and put it in a data frame. Let's look at the first few rows using the `.head()` function that works on all **pandas** data frames.

In [ ]:
flights.head()

To get more general information on the columns, the data types (`dtypes`) of the columns, and the size of the dataset, use `.info()`.

In [ ]:
flights.info()

You might have noticed the short abbreviations that appear in the `Dtypes` column. These tell you the type of the values in their respective columns: `int64` is short for integer (eg whole numbers) and `float64` is short for double-precision floating point number (these are real numbers). `object` is a bit of a catch all category for any data type that **pandas** is not really confident about inferring. Although not found here, other data types include `string` for text and `datetime` for combinations of a date and time.

The different column data types are important because the operations you can perform on a column depend so much on its "type"; for example, you can remove all punctuation from strings while you can multiply ints and floats.

We would like to work with the `"time_hour"` variable in the form of a datetime; fortunately, **pandas** makes it easy to perform that conversion on that specific column

In [ ]:
flights["time_hour"] = pd.to_datetime(flights["time_hour"], format = "%Y-%m-%d %H:%M:%S")

## **pandas** basics

**pandas** is a really comprehensive package, and this book will barely scratch the surface of what it can do. But it's built around a few simple ideas that, once they've clicked, make life a lot easier.

Let’s start with the absolute basics. The most basic pandas object is a dataframe. A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, categorical data, even lists) in columns. It is made up of rows and columns (with each row-column cell containing a value), plus two bits of contextual information: the index (which carries information about each row) and the column names (which carry information about each column).

![](https://pandas.pydata.org/docs/_images/01_table_dataframe.svg)

Perhaps the most important notion to have about **pandas** data frames is that they are built around an index that sits on the left-hand side of the data frame. Every time you perform an operation on a data frame, you need to think about how it might or might not affect the index; or, put another way, whether you want to modify the index.

Let's see a simple example of this with a made-up data frame:

In [ ]:
df = pd.DataFrame(
    data={
        "col0": [0, 0, 0, 0],
        "col1": [0, 0, 0, 0],
        "col2": [0, 0, 0, 0],
        "col3": ["a", "b", "b", "a"],
        "col4": ["alpha", "gamma", "gamma", "gamma"],
    },
    index=["row" + str(i) for i in range(4)],
)
df.head()

You can see there are 5 columns (named `"col0"` to `"col4"`) and that the index consists of four entries named `"row0"` to `"row3"`.

A second key point you should know is that the operations on a **pandas** data frame can be chained together. We need not perform one assignment per line of code; we can actually do multiple assignments in a single command.

Let's see an example of this. We're going to string together four operations:

1. we will use `query` to find only the rows where the destination `"dest"` column has the value `"IAH"`. This doesn't change the index, it only removes irrelevant rows. In effect, this step removes rows we're not interested in.
2. we will use `groupby` to group rows by the year, month, and day (we pass a list of columns to the `groupby` function). This step changes the index; the new index will have three columns in that track the year, month, and day. In effect, this step changes the index.
3. we will choose which columns we wish to keep after the groupby operation by passing a list of them to a set of square brackets (the double brackets are because it's a list within a dataframe). Here we just want one column, `"arr_delay"`. This doesn't affect the index. In effect, this step removes columns we're not interested in.
4. finally, we must specify what groupby operation we wish to apply; when aggregating the information in multiple rows down to one row, we need to say how that information should be aggregated. In this case, we'll use the mean. In effect, this step applies a statistic to the variable(s) we selected earlier, across the groups we created earlier.

In [ ]:
(
    flights.query("dest == 'IAH'")
           .groupby(["year", "month", "day"])
           [["arr_delay"]]
           .mean()
)

You can see here that we've created a new dataframe with a new index. To do it, we used four key operations:

1. manipulating rows
2. manipulating the index
3. manipulating columns
4. applying statistics

Most operations you could want to do to a single dataframe are covered by these, but there are different options for each of them depending on what you need.

Let's now dig a bit more into these operations.

### Manipulating Rows

Let's create some fake data to show how this works.

In [ ]:
import numpy as np

df = pd.DataFrame(
    data=np.reshape(range(36), (6, 6)),
    index=["a", "b", "c", "d", "e", "f"],
    columns=["col" + str(i) for i in range(6)],
    dtype=float,
)
df["col6"] = ["apple", "orange", "pineapple", "mango", "kiwi", "lemon"]
df

#### Accessing Rows

To access a particular row directly, you can use `df.loc['rowname']` or `df.loc[['rowname1', 'rowname1']]` for two different rows.

For example,

In [ ]:
df.loc[["a", "b"]]

But you can also access particular rows based on their location in the dataframe using `.iloc`. Remember that Python indices begin from zero, so to retrieve the first row you would use `.iloc[0]`:


In [ ]:
df.iloc[0]

This works for multiple rows too. Let's grab the first and third rows (in positions 0 and 2) by passing a list of positions:

In [ ]:
df.iloc[[0, 2]]

There are other ways to access multiple rows that make use of *slicing* but we'll leave that topic for another time.

#### Filtering rows with query

As with the flights example, we can also filter rows based on a condition using `query`:

In [ ]:
df.query("col6 == 'kiwi' or col6 == 'pineapple'")

For numbers, you can also use the greater than and less than signs:

In [ ]:
df.query("col0 > 6")

In fact, there are lots of options that work with `query`: as well as `>` (greater than), you can use `>=` (greater than or equal to), `<` (less than), `<=` (less than or equal to), `==` (equal to), and `!=` (not equal to). You can also use the commands `and` as well as `or` to combine multiple conditions. Here's an example of `and` from the `flights` dataframe:

In [ ]:
# Flights that departed on January 1
flights.query("month == 1 and day == 1")

Note that equality is tested by `==` and *not* by `=`, because the latter is used for assignment.

#### Re-arranging Rows

Again and again, you will want to re-order the rows of your dataframe according to the values in a particular column. **pandas** makes this very easy via the `.sort_values` function. It takes a data frame and a set of column names to sort by. If you provide more than one column name, each additional column will be used to break ties in the values of preceding columns. For example, the following code sorts by the departure time, which is spread over four columns.

In [ ]:
flights.sort_values(["year", "month", "day", "dep_time"])

You can use the keyword argument `ascending=False` to re-order by a column or columns in descending order.
For example, this code shows the most delayed flights:

In [ ]:
flights.sort_values("dep_delay", ascending=False)

You can of course combine all of the above row manipulations to solve more complex problems.
For example, we could look for the top three destinations of the flights that were most delayed on arrival that left on roughly on time:

In [ ]:
(
       flights.query("dep_delay <= 10 and dep_delay >= -10")
       .sort_values("arr_delay", ascending=False)
       .iloc[[0, 1, 2]]
       )

#### Exercises

1. Find all flights that
    a. Had an arrival delay of two or more hours
    b. Flew to Houston (`"IAH"` or `"HOU"`)
    c. Were operated by United, American, or Delta
    d. Departed in summer (July, August, and September)
    e. Arrived more than two hours late, but didn't leave late
    f. Were delayed by at least an hour, but made up over 30 minutes in flight

2.  Sort `flights` to find the flights with longest departure delays.

3.  Sort `flights` to find the fastest flights

4.  Which flights traveled the farthest?

5.  Does it matter what order you used `query()` and `sort_values()` in if you're using both? Why/why not? Think about the results and how much work the functions would have to do.

### Manipulating Columns